Transfer learning is leveraging on already existing or pre-trained models for your application

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [2]:
'''
# TODO:Zip file downloade successfully

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

'''

'\n# TODO:Zip file downloade successfully\n\n!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip\n\n'

In [3]:
# import zipfile

# zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip', 'r')
# zip_ref.extractall()
# zip_ref.close()

In [2]:
import os
# for dirnames, filenames, dirpath in os.walk('10_food_classes_10_percent'):
#     print(f"There are {len(filename)} directories and {len(dirname)} files in the '{dirpath}' ")

for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
    print('There are {} files and {} directories in {}'.format(len(filenames), len(dirnames), (dirpath)))

There are 0 files and 2 directories in 10_food_classes_10_percent
There are 0 files and 10 directories in 10_food_classes_10_percent\test
There are 250 files and 0 directories in 10_food_classes_10_percent\test\chicken_curry
There are 250 files and 0 directories in 10_food_classes_10_percent\test\chicken_wings
There are 250 files and 0 directories in 10_food_classes_10_percent\test\fried_rice
There are 250 files and 0 directories in 10_food_classes_10_percent\test\grilled_salmon
There are 250 files and 0 directories in 10_food_classes_10_percent\test\hamburger
There are 250 files and 0 directories in 10_food_classes_10_percent\test\ice_cream
There are 250 files and 0 directories in 10_food_classes_10_percent\test\pizza
There are 250 files and 0 directories in 10_food_classes_10_percent\test\ramen
There are 250 files and 0 directories in 10_food_classes_10_percent\test\steak
There are 250 files and 0 directories in 10_food_classes_10_percent\test\sushi
There are 0 files and 10 directori

Data Preprocessing

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = 224
batch_size = 32

train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'


train_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

print('number of training data')
train_data = train_data_gen.flow_from_directory(train_dir,
                                                target_size = (img_size,img_size),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                shuffle=True)

print('number of testing data')
test_data = test_data_gen.flow_from_directory(test_dir,
                                              target_size = (img_size,img_size),
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)



number of training data
Found 750 images belonging to 10 classes.
number of testing data
Found 2500 images belonging to 10 classes.


Setting up Callbacks
Callbacks are extral functions that you add to your model before or after training
* `TensorBoard callback`
* `model checkpoint callback`
* `Early stopping callback`, etc

In [6]:
import datetime

def creat_tensorboard(dir_name, experiment_name):
    log_dir = dir_name + '/' + experiment_name + datetime.datetime.now().strftime('%Y-%m-%d--%H-%M-%S')
    tensorboard_callback = tf.keras.callbacks.TensorBoardCallback(log_dir)
    print('saving the tensorboard_callback to {}'.format(log_dir))
    return tensorboard_callback 

Transfer learning parameters

In [7]:
import tensorflow_hub as hub
# https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4
# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [4]:
from tensorflow.keras.applications import ResNet50V2, EfficientNetV2B0, VGG16

def create_model(model_instance, num_classes=10):
    model = tf.keras.Sequential()
    for layer in model_instance.layers[:-1]:
        model.add(layer)
    for layer in model.layers:
        layer.trainable = False
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    return model

In [6]:
resnet_model = create_model(VGG16(), num_classes=10)
resnet_model.summary()

278487040/553467096 ━━━━━━━━━━━━━━━━━━━━ 26:56 6us/step

OSError: [Errno 28] No space left on device